In [ ]:
#from textblob import TextBlob
#from attributegetter import *
from generatengrams import ngrammatch
from Contexts import *
import json
from Intents import *
import random
import os
import re
import pandas as pd

In [ ]:
HOTEL_REGEX = "(stay)|(hotel)|(room)|(motel)|(inn)"
RESTAURANT_REGEX = "(eat)|(party)|(table)|(food)|(veg)|(restaurant)|(hungry)|(starving)|(organic)|(cuisine)|(buffet)|(serve)"

In [ ]:
def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent
    as mentioned in the intent config file'''
    '''Performs actions pertaining to current intent'''
    #for action in current_intent.action:
        #print(action)
    
    #print(context)
    #print(type(current_intent.action))
    context = IntentComplete()
    
    if current_intent.action == 'BookRestaurant':
        #print(current_intent.action)
        return BookRestaurant(attributes), context
    if current_intent.action == 'BookHotel':
        return BookHotel(attributes), context

    return 'action: ' + current_intent.action, context

def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    
    for para in current_intent.params:
        if para.required:
            if para.name not in attributes:
                if para.name=='Tariff':
                    context = GetTariff()
                return random.choice(para.prompts), context

    return None, context


def input_processor(user_input, context, attributes, intent):
    '''Spellcheck and entity extraction functions go here'''
    
    #uinput = TextBlob(user_input).correct().string
    
    #update the attributes, abstract over the entities in user input
    attributes, cleaned_input = getattributes(user_input, context, attributes)
    
    return attributes, cleaned_input

def loadIntent(path, intent):
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    scores = ngrammatch(clean_input)
    scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])
    # print clean_input
    #print 'scores', scores
    

    if(current_intent==None):
        if(re.search(RESTAURANT_REGEX, clean_input)):
            return loadIntent('params/newparams.cfg', 'RestaurantBooking')
        if(re.search(HOTEL_REGEX, clean_input)):
            return loadIntent('params/newparams.cfg','HotelBooking')
        else:
            return loadIntent('params/newparams.cfg',scores[-1][0])
    else:
        #print 'same intent'
        return current_intent

def getattributes(uinput,context,attributes):
    '''This function marks the entities in user input, and updates
    the attributes dictionary'''
    #Can use context to to context specific attribute fetching
    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    else:

        files = os.listdir('./entities/')
        entities = {}
        for fil in files:
            lines = open('./entities/'+fil).readlines()
            for i, line in enumerate(lines):
                lines[i] = line[:-1]
            entities[fil[:-4]] = '|'.join(lines)

        for entity in entities:
            for i in entities[entity].split('|'):
                if i.lower() in uinput.lower():
                    attributes[entity] = i
        for entity in entities:
                uinput = re.sub(entities[entity],r'$'+entity,uinput,flags=re.IGNORECASE)

        if context.name=='GetTariff'  and context.active:
            #print('Tariff')
            match = re.findall('[<|>|<=|>=]?\s*[0-9]+\s*[-]?\s*[0-9]+', uinput)
            if len(match):
                uinput = re.sub('[<|>|<=|>=]?\s*[0-9]+\s*[-]?\s*[0-9]+', '$tariff', uinput)
                x=""
                for m in match:
                    x += m + ","
                attributes['Tariff'] = x[:-1]
                context.active = False
        
        print(attributes)
        return attributes, uinput

In [ ]:
class Session:
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):
        
        '''Initialise a default session'''
        
        #Contexts are flags which control dialogue flow, see Contexts.py
        self.active_contexts = active_contexts
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        #self.current_intent = First_Greeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()

    def reply(self, user_input):
        '''Generate response to user input'''
        
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)
        
        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)
        
        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)

        #prompt being None means all parameters satisfied, perform the intent action
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)
        
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete':
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
        
        return prompt

In [ ]:
def BookHotel(attributes):
    #constants
    AMP = "&"
    SPACE = " "
    HOTEL_DB_COL_MAP = {
        "hloc": "LOCATION",
        "starrating": "STAR RATING",
        "Tariff": "TARIFF"
    }
    
    db = pd.read_csv("./db/db_hotel.csv")
    
    for attr, val in attributes.items():
        col_name = HOTEL_DB_COL_MAP[attr]
        #print(val)
        if col_name == "TARIFF":
            val = val.split(",");
            
            for v in val:
                if ">" in v:
                    v = v.replace(">", "").strip()
                    #print(v)
                    db = db[db[col_name] > int(v)]
                if "<" in v:
                    v = v.replace("<", "").strip()
                    db = db[db[col_name] < int(v)]
                if "<=" in v:
                    v = v.replace("<=", "").strip()
                    db = db[db[col_name] <= int(v)]
                if ">=" in v:
                    v = v.replace(">=", "").strip()
                    db = db[db[col_name] >= int(v)]
                if "-" in v:
                    v = v.split("-")
                    db = db[db[col_name] < int(v[0].strip())]
                    db = db[db[col_name] > int(v[1].strip())]
        else:       
            db = db[db[col_name] == val]

    hotel_names = db["HOTEL NAME"]
    output = ["List of Hotels matching your criterion:\n"]
    
    if len(hotel_names):
        for i, hotel in enumerate(hotel_names):
            output.append("{}. {}".format(i + 1, hotel))
        
        return output
    else:
        return "No Hotels found for the provided parameters"
        
def BookRestaurant(attributes):
    #constants
    RESTAURANT_DB_COL_MAP = {
        "rloc": "LOCATION",
        "cuisine": "CUISINE",
        "cost": "COST"
    }
    
    db = pd.read_csv("./db/db_restaurant.csv")
    
    for attr, val in attributes.items():
        col_name = RESTAURANT_DB_COL_MAP[attr]
        db = db[db[col_name] == val]
    
    restaurant_names = db["RESTAURANT NAME"]
    output = ["List of Restaurants matching your criterion:\n"]
    
    if len(restaurant_names):
        for i, restaurant in enumerate(restaurant_names):
            output.append("{}. {}\n".format(i + 1, restaurant))
        
        return output
    else:
        return "No Restaurant found for the provided parameters"
    

In [ ]:
session = Session()

print("RESTAURANT/HOTEL BOOKING PORTAL")
print("Welcome to the Hotel/Restaurant Booking Portal. What do you want to do?")
print("1. Book a Hotel")
print("2. Book a Restaurant\n\n")

print ('[Team 53 BOT]: Hi! How may I assist you?')

while True:
    
    inp = input('User: ')
    print('[User]: ', inp)
    print ('[Team 53 BOT]:', session.reply(inp))